# Enriched PIAAC2ESCO 
We enrich the existing validated mapping by introducing alernative labels and hidden labels as synonyms of the skill that is matched.

## To do:
- add other ESCO Skills that are complement to those matched and enriched

In [1]:
# Libraries
import zipfile
import pandas as pd
import numpy as np
import ast

Import ESCO and prepare the dataset

In [2]:
with zipfile.ZipFile('utils/ESCO dataset - v1.0.3 - classification - en - csv.zip') as z:
   df=pd.read_csv(z.open('skills_en.csv'))

# Create a newcolumn to use hidden labels as preferred labels. 
df['newcol']=df['altLabels']+'\n'+df['hiddenLabels'].astype(str)
df['newcol'].replace({'\nnan':''},regex=True, inplace=True)
# Newcol: From string to list
df['newcol']=df['newcol'].str.split(pat='\n') 

## alternative and hidden lables: From string to list
# df['altLabels']=df['altLabels'].str.split(pat='\n')
# df['hiddenLabels']=df['hiddenLabels'].str.split(pat='\n')

# Select 
df_morelabels=df[['preferredLabel','newcol']]

In [3]:
# # TEST ESCOskill use word processing software
# df_morelabels[df_morelabels['preferredLabel']=='use word processing software'].explode('newcol')

Import the PIAAC-ESCO mapping

In [4]:
df_piaac2esco=pd.read_excel('utils/ESCO_PIAAC_results_mapped_v2.xlsx',usecols=['Isco_Level_2', 'preferredLabel', 'Question Id', 'Question Description','ESCO skill', 'others_skill_rca']).rename(columns={'preferredLabel':'Isco_Level_2_preferredLabel','Question Id':'PIAAC_QuestionId','Question Description':'PIAAC_QuestionDescription','ESCO skill':'ESCOskill','others_skill_rca':'ESCOskill_others'})
# Convert lists from string to list dtypes
df_piaac2esco.loc[:,['ESCOskill','ESCOskill_others']]=df_piaac2esco.loc[:,['ESCOskill','ESCOskill_others']].applymap(lambda x : ast.literal_eval(x))
# Explode the dataset
df_piaac2esco_xlpd=df_piaac2esco.explode('ESCOskill')


Merge the piaac2esco mapping and the enriched set of labels

In [5]:
df_morelabels_xpld=df_morelabels.merge(df_piaac2esco_xlpd,left_on='preferredLabel', right_on='ESCOskill', how='inner').iloc[:,1:-1].rename(columns={'newcol':'ESCOskill'}).explode('ESCOskill')

# concat in final dataset
df_final=pd.concat([df_piaac2esco_xlpd, df_morelabels_xpld])

# Select and polish
df_final=df_final.sort_values(['Isco_Level_2','PIAAC_QuestionId','ESCOskill']).loc[:,['Isco_Level_2','Isco_Level_2_preferredLabel','PIAAC_QuestionId','PIAAC_QuestionDescription','ESCOskill']]

In [124]:
# Export
df_final.to_csv('output/PIAAC2ESCO_enriched.csv', index=False)